HEST

In [1]:
import os
from PIL import Image
Image.MAX_IMAGE_PIXELS = 2800000000
import numpy as np
from cellpose import models

model_cellpose = models.Cellpose(gpu=True, model_type='nuclei')

def process_images(root_dir):
    for subdir, _, files in os.walk(root_dir):
        for file in files:
            if file.endswith('.tif'):
                file_path = os.path.join(subdir, file)
                npy_file_path = os.path.splitext(file_path)[0] + '.npy'
                if os.path.exists(npy_file_path):
                    continue
                if not os.path.exists(file_path.replace('.tif', '.h5ad')):
                    print(file_path)
                    continue

                image = Image.open(file_path)
                image = np.array(image)
                image = image[:,:,0:3]
                print(image.shape)

                nuclei_masks,_,_,_=model_cellpose.eval(image, flow_threshold=0.8, diameter=None, 
                                                   min_size=15,channels=[1,0], invert=True)
                print(nuclei_masks.shape)

                np.save(npy_file_path, nuclei_masks)
                print(f"Processed and saved: {npy_file_path}")

# 使用示例
root_directorys=['./data/HEST/COLON-CANCER/','./data/HEST/COLON-HEALTHY/']
for root_directory in root_directorys:
    process_images(root_directory)